In [11]:
import torch
from src.model import ChessNet
from src.dataclass import ChessDataset

In [12]:
val_dataset = ChessDataset("data/csv/evalA.csv")
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=4)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChessNet()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
checkpoint = torch.load(f"models/test_model.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1

In [15]:
model.to(device)
model.eval()

ChessNet(
  (conv1): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=4544, bias=True)
)

In [6]:
correct_top_1 = 0
correct_top_3 = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probabilities = torch.softmax(outputs, dim=1)

        pred_top_1 = probabilities.argmax(dim=1)
        correct_top_1 += (pred_top_1 == labels).sum().item()

        pred_top_3 = torch.topk(probabilities, k=3, dim=1).indices
        correct_top_3 += sum([labels[i] in pred_top_3[i] for i in range(len(labels))])

        total += labels.size(0)

top_1_accuracy = 100 * correct_top_1 / total
top_3_accuracy = 100 * correct_top_3 / total

In [8]:
print(f"\nValidation Results:\nTop-1 accuracy: {top_1_accuracy:.2f}%\nTop-3 accuracy: {top_3_accuracy:.2f}%")


Validation Results:
Top-1 accuracy: 24.90%
Top-3 accuracy: 44.22%


In [16]:
import random
import chess

In [17]:
import chess
import numpy as np

index_to_piece = {
    0: chess.PAWN, 1: chess.KNIGHT, 2: chess.BISHOP, 3: chess.ROOK, 4: chess.QUEEN, 5: chess.KING,
    6: chess.PAWN, 7: chess.KNIGHT, 8: chess.BISHOP, 9: chess.ROOK, 10: chess.QUEEN, 11: chess.KING,
}

index_to_color = {
    0: chess.WHITE, 1: chess.WHITE, 2: chess.WHITE, 3: chess.WHITE, 4: chess.WHITE, 5: chess.WHITE,
    6: chess.BLACK, 7: chess.BLACK, 8: chess.BLACK, 9: chess.BLACK, 10: chess.BLACK, 11: chess.BLACK,
}

def tensor_to_fen(tensor, default_fen_info=" w KQkq - 0 1"):
    """
    Convert a (12,8,8) tensor back into a FEN string.

    Args:
        tensor (np.ndarray): shape (12,8,8), as created by fen_to_tensor.
        default_fen_info (str): provides side to move, castling, etc., since your tensor doesn't store them.

    Returns:
        fen (str): reconstructed FEN string.
    """
    board = chess.Board(fen=None)  # create empty board
    board.clear()  # remove any existing pieces

    for idx in range(12):
        piece_type = index_to_piece[idx]
        color = index_to_color[idx]
        for row in range(8):
            for col in range(8):
                if tensor[idx, row, col] > 0.5:  # treat as "piece exists"
                    rank = 7 - row  # inverse of your encoding: row=0 is rank 8
                    square = chess.square(col, rank)
                    piece = chess.Piece(piece_type, color)
                    board.set_piece_at(square, piece)

    fen = board.board_fen() + default_fen_info
    return fen



In [18]:
from src.encode import index_to_uci
model.eval()
with torch.no_grad():
    for i in range(5):
        fen, label = val_dataset[random.randint(0, len(val_dataset)-1)]
        board = chess.Board(tensor_to_fen(fen))
        input_tensor = torch.tensor(fen).unsqueeze(0).to(device)
        probs = torch.softmax(model(input_tensor), dim=1).squeeze()
        topk = torch.topk(probs, k=3)

        print(f"\nSample {i+1}:")
        print(board.unicode())
        print("Top-3 predicted moves:")
        for idx, prob in zip(topk.indices, topk.values):
            print(f"  {index_to_uci[idx.item()]} - {prob.item():.4f}")
        print(f"True move: {index_to_uci[label.item()]}")


C:\Users\myPC\AppData\Local\Temp\ipykernel_9940\3600445335.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(fen).unsqueeze(0).to(device)



Sample 1:
⭘ ♞ ⭘ ♜ ⭘ ♜ ♚ ⭘
⭘ ♟ ♟ ⭘ ♛ ♟ ♟ ♟
♟ ⭘ ⭘ ⭘ ⭘ ♝ ♝ ⭘
⭘ ⭘ ♙ ♟ ⭘ ⭘ ⭘ ⭘
♙ ♙ ⭘ ♙ ⭘ ⭘ ⭘ ⭘
⭘ ♕ ⭘ ⭘ ♗ ♘ ⭘ ♙
⭘ ⭘ ⭘ ⭘ ♗ ♙ ♙ ⭘
⭘ ⭘ ♖ ♖ ⭘ ⭘ ♔ ⭘
Top-3 predicted moves:
  b4b5 - 0.1934
  c7c6 - 0.0950
  f8e8 - 0.0691
True move: c7c6

Sample 2:
♜ ⭘ ⭘ ⭘ ♜ ⭘ ♚ ⭘
⭘ ⭘ ♝ ♛ ⭘ ♟ ♟ ♝
⭘ ♟ ⭘ ⭘ ⭘ ♞ ⭘ ♟
♟ ⭘ ♟ ⭘ ♟ ♙ ⭘ ⭘
♙ ⭘ ♙ ♟ ♘ ⭘ ⭘ ⭘
♖ ⭘ ⭘ ♙ ⭘ ♕ ⭘ ♙
⭘ ♙ ♗ ♗ ⭘ ♙ ♙ ♔
⭘ ⭘ ⭘ ⭘ ♖ ⭘ ⭘ ⭘
Top-3 predicted moves:
  f6e4 - 0.2153
  e4f6 - 0.1261
  g2g4 - 0.0731
True move: c7d8

Sample 3:
♜ ♞ ♝ ♛ ♜ ⭘ ♚ ⭘
♟ ♟ ⭘ ♞ ⭘ ♟ ♝ ♟
⭘ ⭘ ⭘ ♟ ♟ ⭘ ♟ ⭘
⭘ ⭘ ♟ ♙ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ♙ ⭘ ♙ ⭘ ♙ ♙
⭘ ⭘ ♘ ⭘ ♗ ⭘ ⭘ ⭘
♙ ♙ ⭘ ♕ ♗ ♙ ⭘ ⭘
♖ ⭘ ⭘ ⭘ ♔ ⭘ ♘ ♖
Top-3 predicted moves:
  b8a6 - 0.2805
  a7a6 - 0.2020
  d7e5 - 0.1336
True move: e6d5

Sample 4:
♜ ⭘ ⭘ ⭘ ♚ ⭘ ⭘ ♜
⭘ ♟ ⭘ ♞ ♝ ♟ ♟ ⭘
⭘ ♛ ♟ ⭘ ♟ ⭘ ⭘ ♟
♟ ⭘ ⭘ ♟ ♙ ♞ ⭘ ⭘
♙ ⭘ ⭘ ♙ ⭘ ♗ ⭘ ⭘
⭘ ⭘ ♙ ⭘ ⭘ ♘ ⭘ ♙
⭘ ♙ ⭘ ♕ ♗ ♙ ♙ ⭘
♖ ⭘ ⭘ ⭘ ⭘ ♖ ♔ ⭘
Top-3 predicted moves:
  g2g4 - 0.3038
  g7g5 - 0.1463
  c6c5 - 0.0606
True move: f4h2

Sample 5:
♜ ♞ ♝ ♛ ♚ ♝ ⭘ ♜
♟ ♟ ♟ ⭘ ♟ ♟ ⭘ ♟
⭘ ⭘ ⭘ ⭘ ⭘ ♞ ♟ ⭘
⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘ ⭘
⭘ ⭘ ⭘ ♙ ♙ ⭘ ⭘ ⭘
⭘ ⭘ ⭘

In [19]:
from src.encode import fen_to_tensor
def predict_move(board):
    fen_tensor = torch.from_numpy(fen_to_tensor(board.fen())).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(fen_tensor)
        probs = torch.softmax(logits, dim=1).squeeze()
        top_moves = torch.topk(probs, k=20)
        for idx in top_moves.indices:
            uci_move = index_to_uci[idx.item()]
            move = chess.Move.from_uci(uci_move)
            if move in board.legal_moves:
                return move
    return None

In [24]:
import chess.pgn

game = chess.pgn.Game()
game.headers["Event"] = "Evaluation Game"
game.headers["Site"] = "Local"
game.headers["White"] = "Human"
game.headers["Black"] = "Model"

node = game

board = chess.Board()
while not board.is_game_over():
    print(board)
    if board.turn == chess.WHITE:
        print("\nModel (White) is thinking...")
        model_move = predict_move(board)
        if model_move and model_move in board.legal_moves:
            print(f"Model plays: {model_move.uci()}")
            board.push(model_move)
            node.add_variation(model_move)
        else:
            print("No legal moves found by model.")
            break
    else:
        print("\nModel (Black) is thinking...")
        model_move = predict_move(board)
        if model_move  and model_move in board.legal_moves:
            print(f"Model plays: {model_move.uci()}")
            board.push(model_move)
            node.add_variation(model_move)
        else:
            print("No legal moves found by model.")
            break
print("\nGame over:", board.result())
with open("game_output.pgn", "w") as pgn_file:
    print(game, file=pgn_file)

print("Game saved to game_output.pgn — upload it to Lichess to review!")

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R

Model (White) is thinking...
Model plays: e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

Model (Black) is thinking...
Model plays: c7c5
r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R

Model (White) is thinking...
Model plays: g1f3
r n b q k b n r
p p . p p p p p
. . . . . . . .
. . p . . . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K B . R

Model (Black) is thinking...
Model plays: b8c6
r . b q k b n r
p p . p p p p p
. . n . . . . .
. . p . . . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K B . R

Model (White) is thinking...
Model plays: d2d4
r . b q k b n r
p p . p p p p p
. . n . . . . .
. . p . . . . .
. . . P P . . .
. . . . . N . .
P P P . . P P P
R N B Q 

AssertionError: san() and lan() expect move to be legal or null, but got c5d4 in rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1